In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/jigsaw-unintended-bias-in-toxicity-classification/sample_submission.csv
/kaggle/input/jigsaw-unintended-bias-in-toxicity-classification/all_data.csv
/kaggle/input/jigsaw-unintended-bias-in-toxicity-classification/test_public_expanded.csv
/kaggle/input/jigsaw-unintended-bias-in-toxicity-classification/test_private_expanded.csv
/kaggle/input/jigsaw-unintended-bias-in-toxicity-classification/toxicity_individual_annotations.csv
/kaggle/input/jigsaw-unintended-bias-in-toxicity-classification/train.csv
/kaggle/input/jigsaw-unintended-bias-in-toxicity-classification/identity_individual_annotations.csv
/kaggle/input/jigsaw-unintended-bias-in-toxicity-classification/test.csv


In [2]:
train =pd.read_csv('/kaggle/input/jigsaw-unintended-bias-in-toxicity-classification/train.csv')
test = pd.read_csv('/kaggle/input/jigsaw-unintended-bias-in-toxicity-classification/test.csv')
df_test = test['comment_text']
df = train[['target','comment_text']]

In [3]:
from imblearn.under_sampling import RandomUnderSampler
X = df['comment_text']
X = X.values.reshape(-1,1)
y = df['target']
y = np.where(y>=0.5,1.,0.)
undersample = RandomUnderSampler(sampling_strategy='majority')
X_sample, y_sample = undersample.fit_resample(X, y)

In [4]:
unique, counts = np.unique(y_sample, return_counts=True)
unique, counts

(array([0., 1.]), array([144334, 144334]))

In [5]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X_sample.reshape(-1), y_sample, 
                                                    test_size=0.3, random_state=42)

In [6]:
!pip install -U sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.7/79.7 kB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.0-py3-none-any.whl size=120747 sha256=d24f3c0b16c90171108e62771894c1f38521f3ed70d8335cfffcddb5532c3c17
  Stored in directory: /root/.cache/pip/wheels/83/c0/df/b6873ab7aac3f2465aa9144b6b4c41c4391cfecc027c8b07e7
Successfully built sentence-transformers


In [7]:
from sentence_transformers import SentenceTransformer
model_trans = SentenceTransformer('all-MiniLM-L6-v2')

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/612 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/350 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

In [8]:
sentences = ["The company HuggingFace is based in New York City"]

sentence_embeddings = model_trans.encode(sentences)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [9]:
sentence_embeddings.shape

(1, 384)

In [10]:
x_train_v= np.zeros((len(x_train),384))
x_test_v= np.zeros((len(x_test),384))
x_train_v = model_trans.encode(x_train)

Batches:   0%|          | 0/6315 [00:00<?, ?it/s]

In [11]:
x_test_v = model_trans.encode(x_test)

Batches:   0%|          | 0/2707 [00:00<?, ?it/s]

In [12]:
from keras.models import Sequential
from keras.layers import Dense
model = Sequential()
model.add(Dense(units = 50, kernel_initializer = 'uniform', activation = 'relu', 
                input_dim = x_train_v.shape[1]))
model.add(Dense(units = 25, kernel_initializer = 'uniform', activation = 'relu'))
model.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 50)                19250     
_________________________________________________________________
dense_1 (Dense)              (None, 25)                1275      
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 26        
Total params: 20,551
Trainable params: 20,551
Non-trainable params: 0
_________________________________________________________________


2022-06-03 00:35:15.886066: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [13]:
model.fit(x_train_v, y_train,batch_size=32,epochs=20,validation_data=(x_test_v,y_test))

2022-06-03 00:35:16.372664: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/20
6315/6315 [==============================] - 17s 3ms/step - loss: 0.4656 - accuracy: 0.7786 - val_loss: 0.4519 - val_accuracy: 0.7849
Epoch 2/20
6315/6315 [==============================] - 15s 2ms/step - loss: 0.4402 - accuracy: 0.7929 - val_loss: 0.4355 - val_accuracy: 0.7957
Epoch 3/20
6315/6315 [==============================] - 14s 2ms/step - loss: 0.4259 - accuracy: 0.8019 - val_loss: 0.4287 - val_accuracy: 0.8006
Epoch 4/20
6315/6315 [==============================] - 14s 2ms/step - loss: 0.4168 - accuracy: 0.8076 - val_loss: 0.4254 - val_accuracy: 0.8020
Epoch 5/20
6315/6315 [==============================] - 14s 2ms/step - loss: 0.4091 - accuracy: 0.8124 - val_loss: 0.4241 - val_accuracy: 0.8023
Epoch 6/20
6315/6315 [==============================] - 15s 2ms/step - loss: 0.4024 - accuracy: 0.8154 - val_loss: 0.4218 - val_accuracy: 0.8041
Epoch 7/20
6315/6315 [==============================] - 14s 2ms/step - loss: 0.3964 - accuracy: 0.8186 - val_loss: 0.4231 - val_ac

In [14]:
model.evaluate(x_test_v, y_test)

2707/2707 [==============================] - 3s 1ms/step - loss: 0.4500 - accuracy: 0.7980


[0.450018048286438, 0.7979815602302551]

In [15]:
test_df= pd.read_csv('/kaggle/input/jigsaw-unintended-bias-in-toxicity-classification/test.csv')
test_df= test_df['comment_text']
test_df.head()

0    [ Integrity means that you pay your debts.]\n\...
1    This is malfeasance by the Administrator and t...
2    @Rmiller101 - Spoken like a true elitist. But ...
3    Paul: Thank you for your kind words.  I do, in...
4    Sorry you missed high school. Eisenhower sent ...
Name: comment_text, dtype: object

In [16]:
test_data =  model_trans.encode(test_df)

Batches:   0%|          | 0/3042 [00:00<?, ?it/s]

In [17]:
y_pred= model.predict(test_data)
y_pred = np.where(y_pred>=0.5,1.,0.)
sub_df = pd.read_csv('/kaggle/input/jigsaw-unintended-bias-in-toxicity-classification/sample_submission.csv')
sub_df['prediction'] = y_pred
sub_df.to_csv('submission.csv',index = False)